# Algoritmi linearnih parametrizacije 3D modela

Koristimo python pakete libigl, numpy, scipy, te meshplot:

In [1]:
import igl
import scipy as sp
import numpy as np
from meshplot import plot, subplot, interact

import scipy.spatial
from scipy.linalg import block_diag
from scipy.sparse import issparse
import scipy.sparse.linalg as sla
from scipy.sparse import csc_matrix


import scipy.linalg as la

import os
root_folder = os.getcwd()

Loadamo mrežu koju želimo parametrizirati:

In [2]:
V, F  = igl.read_triangle_mesh(os.path.join(root_folder, "camelhead.off"))

## Harmonijsko preslikavanje

In [4]:
    b = igl.boundary_loop(F)

    bc = igl.map_vertices_to_circle(V,b)

    L = igl.cotmatrix(V,F)

    Q = -L
    
    B=np.zeros((V.shape[0],2))
    Aeq = sp.sparse.csc_matrix(np.zeros((1,V.shape[0])))
    Beq = np.zeros((1,2))
    
    _, Harmonic_uv=igl.min_quad_with_fixed(Q, B, b, bc, Aeq, Beq, False)


## Baricentrično konveksno preslikavanje

In [30]:
    
    # Find boundary vertices
    b = igl.boundary_loop(F)
    # Map them to circle
    bc = igl.map_vertices_to_circle(V,b)
    
    # List interior & boundary vertices
    e = igl.boundary_facets(F)
    v_b = np.unique(e)
    v_all = np.arange(V.shape[0])

    # List of interior indices
    v_in = np.setdiff1d(v_all, b)

    # Adjacency matrix for given mesh
    a = igl.adjacency_matrix(F)
    # Sum each row to get diagonal
    a_sum = np.sum(a, axis=1)
    a_sumsum = np.array(a_sum).ravel()
    # Convert row sums into diagonal of sparse matrix
    a_diag = np.diag(a_sumsum)
    # Build uniform laplacian of graph
    u = a - a_diag
    
    # Prepare system matrix
    l_ii = u[v_in, :]
    l_ii = l_ii[:, v_in]
    l_ib = u[v_in, :]
    l_ib = l_ib[:, b]
    
    ## Solve PDE
    xs = sla.spsolve(-l_ii, np.transpose(l_ib.dot(bc[:,0])))
    ys = sla.spsolve(-l_ii, np.transpose(l_ib.dot(bc[:,1])))
    uv= np.column_stack((xs,ys))
    
    # Write vertices in correct order
    Barycentric_uv= np.zeros((V.shape[0],2))
    Barycentric_uv[v_in, :] = uv
    Barycentric_uv[b, :] = bc

C:\Users\Davorin\anaconda3\lib\site-packages\scipy\sparse\linalg\dsolve\linsolve.py:138: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  SparseEfficiencyWarning)


## Konveksno preslikavanje srednje vrijednosti

In [35]:
    # Find boundary vertices
    b = igl.boundary_loop(F)
    # Map them to circle
    bc = igl.map_vertices_to_circle(V,b)
    
    # List interior & boundary vertices
    e = igl.boundary_facets(F)
    v_b = np.unique(e)
    v_all = np.arange(V.shape[0])

    # List of interior indices
    v_in = np.setdiff1d(v_all, b)
    
    # Find distance between all points in the mesh
    distances = igl.all_pairs_distances(V,V, False)
    # Find angle values for each triangle in a mesh
    angles = igl.internal_angles(V,F)
    
    # Prob not needed
    tt, tti = igl.triangle_triangle_adjacency(F)
    neighbours = igl.adjacency_matrix(F)
    
    # Create edge topology, edges-vertices, faces-edges, edges-faces:
    ev, fe, ef = igl.edge_topology(V,F)
    
    # Initialize system matrix
    weights = np.zeros((V.shape[0],V.shape[0]))
    
    # For given list of triangles and a vertex, find edges opposite vertex v
    def find_edges(triangles, vertex, ev):
        target_edges = []
        for triangle in triangles:
            for edge in triangle:
                if vertex not in ev[edge]:
                    target_edges = np.append(target_edges, edge)
        return target_edges
    
    # Start building system matrix
    for i in range(ev.shape[0]):
        v_i, v_j = ev[i][0], ev[i][1]
        if ((v_i not in b) or (v_j not in b)):
            distance = distances[v_i,v_j]
            # Triangles that belong to i-th edge: ef[i,:]
            # All edges in those triangles:
            all_edges = fe[ef[i,:],:]
            # Angles inside those triangles
            relevant_angles = angles[ef[i,:],]
            # Of those edges, find the one that v_i doesn't belong to
            #(the one opposite v_i)
            relevant_edges_i = find_edges(all_edges, v_i, ev)
            # Find the angles
            alphas_i=relevant_angles[np.isin(all_edges,relevant_edges_i)]
            # Add to system matrix
            weights[v_i,v_j] = (np.tan(0.5*alphas_i[0])+np.tan(0.5*alphas_i[1]))/distance
            # Do the same for edge in opposite direction
            relevant_edges_j = find_edges(all_edges, v_j, ev)
            alphas_j=relevant_angles[np.isin(all_edges,relevant_edges_j)]
            weights[v_j,v_i] = (np.tan(0.5*alphas_j[0])+np.tan(0.5*alphas_j[1]))/distance
    
    # Add diagonal to system matrix
    w_sum = np.sum(weights, axis=1)
    w_sumsum = np.array(w_sum).ravel()
    w_diag = np.diag(w_sumsum)
    u = weights - w_diag
    
    # Prepare system matrix for efficient solve
    l_ii = u[v_in, :]
    l_ii = l_ii[:, v_in]
    l_ib = u[v_in, :]
    l_ib = l_ib[:, b]
    
    ## Solve PDE
    xs = sla.spsolve(-l_ii, np.transpose(l_ib.dot(bc[:,0])))
    ys = sla.spsolve(-l_ii, np.transpose(l_ib.dot(bc[:,1])))
    uv= np.column_stack((xs,ys))
    
    # Write vertices in correct order
    Convex_uv= np.zeros((V.shape[0],2))
    Convex_uv[v_in, :] = uv
    Convex_uv[b, :] = bc

## Konformalno preslikavanje najmanjih kvadrata

In [78]:

b = np.array([2, 1])
bnd = igl.boundary_loop(F)
points = np.take(V, bnd, axis=0)

# Max distance between points
D = sp.spatial.distance.pdist(points)
D = sp.spatial.distance.squareform(D);
N, [I_row, I_col] = np.nanmax(D), np.unravel_index( np.argmax(D), D.shape )

# Random points
#b[0] = bnd[0]
#b[1] = bnd[int(bnd.size / 2 )]

#maxdistance
b[0] = bnd[I_row]
b[1] = bnd[I_col]


bc = np.array([[0.0, 0.0], [1.0, 0.0]])

# LSCM parametrization
_, LSCM_uv = igl.lscm(V, F, b, bc)   

## Spektralno konformalno preslikavanje

In [12]:
    def block_diag(*arrs):
    
        if arrs == ():
            arrs = ([],)

        arrs = [a.todense() for a in arrs]
        arrs = [np.atleast_2d(a) for a in arrs]
        bad_args = [k for k in range(len(arrs)) if arrs[k].ndim > 2]
        if bad_args:
            raise ValueError("arguments in the following positions have dimension "
                             "greater than 2: %s" % bad_args)
        shapes = np.array([a.shape for a in arrs])
        out_dtype = np.find_common_type([arr.dtype for arr in arrs], [])
        out = np.zeros(np.sum(shapes, axis=0), dtype=out_dtype)

        r, c = 0, 0
        for i, (rr, cc) in enumerate(shapes):
            out[r:r + rr, c:c + cc] = arrs[i]
            r += rr
            c += cc
        return out

    A = igl.vector_area_matrix(F)
    L = igl.cotmatrix(V, F)
    L_diag = block_diag(L,L)
    L_diag_sparse = csc_matrix(L_diag)
    
    
    Q = csc_matrix(L_diag_sparse) + csc_matrix(2 * A)
    
    M=igl.massmatrix(V, F, igl.MASSMATRIX_TYPE_VORONOI)

    B = block_diag(M,M)
    
    # Bricks here
    v, lamb = sla.eigs(Q, 3, csc_matrix(B), which='SM')
    
    U = np.zeros(V[0],2)
    U[:,0] = v[:V[0].size,1]
    U[:,1] = v[V[0].size:,1]
    
    u_1, s_1, v_1 = sla.svd(U * np.transpose(U), full_matrices=False)
    
    U = U*u_1


KeyboardInterrupt: 

## Vizualizacija rezultata

In [70]:
p = plot(V, F, return_plot=True)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(1.9967555…

### Harmonijsko

In [26]:
p0 = plot(V, F, return_plot=True)

@interact(mode=['3D (originalan model)','2D (parametrizacija)', '2D (uv-izolinije)', '3D (s uv-izolinijama)'])
def switch(mode):
    if mode == "3D (originalan model)":
        plot(V, F, shading={"wireframe": True, "flat": False}, plot=p0)
    if mode == "2D (parametrizacija)":
        plot(Harmonic_uv, F, shading={"wireframe": True}, plot=p0)
    if mode == "2D (uv-izolinije)":
        plot(Harmonic_uv, F, uv=Harmonic_uv, shading={"wireframe": False}, plot=p0)
    if mode == "3D (s uv-izolinijama)":
        plot(V, F, uv=Harmonic_uv, shading={"wireframe": False, "flat": False}, plot=p0)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(1.9967555…

interactive(children=(Dropdown(description='mode', options=('3D (originalan model)', '2D (parametrizacija)', '…

### Baricentrične koodrinate

In [31]:
p1 = plot(V, F, return_plot=True)

@interact(mode=['3D (originalan model)','2D (parametrizacija)', '2D (uv-izolinije)', '3D (s uv-izolinijama)'])
def switch(mode):
    if mode == "3D (originalan model)":
        plot(V, F, shading={"wireframe": True, "flat": False}, plot=p1)
    if mode == "2D (parametrizacija)":
        plot(Barycentric_uv, F, shading={"wireframe": True}, plot=p1)
    if mode == "2D (uv-izolinije)":
        plot(Barycentric_uv, F, uv=Barycentric_uv, shading={"wireframe": False}, plot=p1)
    if mode == "3D (s uv-izolinijama)":
        plot(V, F, uv=Barycentric_uv, shading={"wireframe": False, "flat": False}, plot=p1)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(1.9967555…

interactive(children=(Dropdown(description='mode', options=('3D (originalan model)', '2D (parametrizacija)', '…

### Konveksno preslikavanje srednje vrijednosti

In [72]:
p2 = plot(V, F, return_plot=True)

@interact(mode=['3D (originalan model)','2D (parametrizacija)', '2D (uv-izolinije)', '3D (s uv-izolinijama)'])
def switch(mode):
    if mode == "3D (originalan model)":
        plot(V, F, shading={"wireframe": True, "flat": False}, plot=p2)
    if mode == "2D (parametrizacija)":
        plot(Convex_uv, F, shading={"wireframe": True}, plot=p2)
    if mode == "2D (uv-izolinije)":
        plot(Convex_uv, F, uv=Convex_uv, shading={"wireframe": False}, plot=p2)
    if mode == "3D (s uv-izolinijama)":
        plot(V, F, uv=Convex_uv, shading={"wireframe": False, "flat": False}, plot=p2)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(1.9967555…

interactive(children=(Dropdown(description='mode', options=('3D (originalan model)', '2D (parametrizacija)', '…

### Konformalno preslikavanje najmanjih kvadrata

In [79]:
p3 = plot(V, F, return_plot=True)

@interact(mode=['3D (originalan model)','2D (parametrizacija)', '2D (uv-izolinije)', '3D (s uv-izolinijama)'])
def switch(mode):
    if mode == "3D (originalan model)":
        plot(V, F, shading={"wireframe": True, "flat": False}, plot=p3)
    if mode == "2D (parametrizacija)":
        plot(LSCM_uv, F, shading={"wireframe": True}, plot=p3)
        p3.add_points( LSCM_uv[b[0]], shading={"point_color": "red", "point_size": .1})
        p3.add_points( LSCM_uv[b[1]], shading={"point_color": "red", "point_size": .1})
    if mode == "2D (uv-izolinije)":
        plot(LSCM_uv, F, uv=LSCM_uv, shading={"wireframe": False}, plot=p3)
        p3.add_points( LSCM_uv[b[0]], shading={"point_color": "red", "point_size": .1})
        p3.add_points( LSCM_uv[b[1]], shading={"point_color": "red", "point_size": .1})
    if mode == "3D (s uv-izolinijama)":
        plot(V, F, uv=LSCM_uv, shading={"wireframe": False, "flat": False}, plot=p3)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(1.9967555…

interactive(children=(Dropdown(description='mode', options=('3D (originalan model)', '2D (parametrizacija)', '…

### Spektralno konformalno preslikavanje

In [24]:
print("SOON")

SOON


# Mjere

### Flipped triangles

In [118]:
igl.flipped_triangles(Harmonic_uv,F).size

0

 $\rightarrow$ nema okrenutih trokuta

In [119]:
igl.flipped_triangles(Barycentric_uv,F).size

0

In [120]:
igl.flipped_triangles(Convex_uv,F).size

0

In [121]:
igl.flipped_triangles(LSCM_uv,F).size

0

### Kvaliteta trokuta

In [113]:
def quality_fun(angles):
    return 4*(np.sin(angles[0])*np.sin(angles[1])*np.sin(angles[2]))/(np.sin(angles[0]) + np.sin(angles[1]) + np.sin(angles[2]))

def min_mean(angle_list):
    results = []
    for angles in angle_list:
        results = np.append(results, quality_fun(angles))
    return np.min(results), np.mean(results)

In [122]:
Harmonic_angles = igl.internal_angles(Harmonic_uv,F)

print(min_mean(Harmonic_angles))

(0.0008613340789933834, 0.6766186732758929)


In [115]:
Barycentric_angles = igl.internal_angles(Barycentric_uv,F)

print(min_mean(Barycentric_angles))

(0.375128256494023, 0.9846939499607019)


In [116]:
Convex_angles = igl.internal_angles(Convex_uv,F)

print(min_mean(Convex_angles))

(0.00017903454576607587, 0.6929539891571794)


In [117]:
LSCM_angles = igl.internal_angles(LSCM_uv,F)

print(min_mean(LSCM_angles))

(0.0008294948403650462, 0.6761743895985035)
